In [5]:
import pandas as pd
import json

with open('/Users/yasminezaatour/Desktop/tunisia-stock-market/stock_scraper/companies_data/AB.json') as f:
    data = json.load(f)

df = pd.json_normalize(data, 'QuoteTab', ['Name'])

# Rename the columns
df.rename(columns={
    'Name': 'companyName',
    'd': 'date',
    'o': 'openingPrice',
    'h': 'highestPrice',
    'l': 'lowestPrice',
    'c': 'closingPrice',
    'v': 'volume'
}, inplace=True)
df

,date,openingPrice,highestPrice,lowestPrice,closingPrice,volume,companyName
0,16/06/2014,23.63,23.63,22.75,23.14,1608,AMEN BANK
1,23/06/2014,23.14,23.14,22.37,22.75,16837,AMEN BANK
2,30/06/2014,22.75,22.97,22.07,22.66,33514,AMEN BANK
3,07/07/2014,22.75,23.17,21.88,22.74,3340,AMEN BANK
4,14/07/2014,22.84,23.16,22.58,22.75,5789,AMEN BANK
...,...,...,...,...,...,...,...
517,13/05/2024,40.00,41.12,39.50,39.99,99239,AMEN BANK
518,20/05/2024,39.99,41.12,39.99,40.90,44343,AMEN BANK
519,27/05/2024,40.95,41.50,40.42,41.18,46829,AMEN BANK
520,03/06/2024,41.18,42.00,41.00,42.00,24726,AMEN BANK


In [4]:
print(df.shape)


(522, 7)
